# Parsing a CS2 Demofile

To download awpy, run the following in your Terminal/Command Prompt window:

`
pip install awpy
`

### What is a demofile?

Counter-Strike 2 (CS2) games can be recorded through a _demofile_. Every map in a match will generate its own demofile. This demofile contains a serialization of the data transferred between the host (game server) and its clients (players). Data transferred to and from the server occurs at a predefined tick rate, which defines when client inputs are resolved with the server. For professional games, the server tick rate is usually 128 ticks per second, meaning each tick represents around 7.8 milliseconds. Client inputs represent player actions, such as movement, attacks or grenade throws. Non-client events also register in the demofile, such as round starts and ends. Awpy allows you to parse the rich data that exists within CS2 demofiles.

### How do I get a demofile?

HLTV, FACEIT, and ESEA are common sources for demofiles. HLTV hosts most professional and semi-pro CS2 tournament demos, FACEIT hosts organized pickup games, and ESEA generally hosts organized gameplay.

Let's consider the demo from a match between Natus Vincere (NaVi) and Virtus Pro (VP), which we can download from [HLTV](https://www.hltv.org/matches/2369248/natus-vincere-vs-virtuspro-pgl-cs2-major-copenhagen-2024-europe-rmr-closed-qualifier-a). After navigating to the match's page on HLTV, find the relevant button within the `Rewatch` section to download the demofile. If we download the compressed demofile directory from the previous mentioned game (a few hundred MB), there are two files (one demo file for each of the maps played during the match):

* natus-vincere-vs-virtus-pro-m1-overpass.dem
* natus-vincere-vs-virtus-pro-m2-anubis.dem

If you are interested in learning how to access demofiles through Faceit, please take a look at [this article](https://support.faceit.com/hc/en-us/articles/10622392832412-How-to-download-and-watch-a-CS2-demo#:~:text=Click%20on%20'Watch%20Demo'%20at,your%20preferred%20file%20decompression%20software) which should give some context.

## Parsing Demos With Awpy

In order to parse one of the demofiles, you have to use the demoparser.parse_demo method within the awpy.parser module and pass it the path to the demofile." -> "To parse a CS2 demo, you only need to use the `parse_demo` function from `awpy.parser`. To use `parse_demo`, simply pass the the demo via `parse_demo(file="...")`. There are also a few optional arguments:

* ticks (bool): Optional argument (default is `True`) on whether to parse tick-level data. If set as true, ticks attribute of returned demo object will contain a DataFrame with a row for each player during each tick. More detail on tick-level data can be found later in this notebook (**Ticks** subsection).
* rounds (bool): Optional argument (default is `True`) on whether to parse rounds. If set as True, `round_id` column is added to the DataFrame(s) in demo.events and demo.ticks (if applicable). The `round_id` will help identify which round each row of a DataFrame (whether an event or a tick) belongs to.
* extended_ticks (bool): Optional argument (default is `False`) on whether to parse extra information for each tick. Examples of extra information include if a player is currently scoped in, if a player is currently walking, etc. More detail on extended tick-level data can be found later in this notebook (**Ticks** subsection).
* extended_events (bool): Optional argument (default is `False`) on whether to parse extra events. Examples of extra events include if the bomb has been picked up, if the bomb has been dropped, etc. More detail on extended event-level data can be found later in this notebook (**Events** subsection).
* keystrokes (bool): Optional argument (default is `False`) on whether to parse keystrokes in the tick-level data. More detail on keystrokes can be found later in this notebook (**Ticks** subsection).
* custom_events (list[str]): Optional argument (default is `None`) on whether to parse additional events from the demoparser2 library which aren't directly available through awpy. Please refer to the [demoparser2 repo](https://github.com/LaihoE/demoparser) to learn more about these events.

In [14]:
from awpy.parser import demoparser

demo = demoparser.parse_demo(file="natus-vincere-vs-virtus-pro-m1-overpass.dem")

## Accessing the parsed data

The output of `parse_demo` is a `Demo` object (whose definition can be found in `awpy.parser.models.demo`). The demo object has the following attributes: header, events, ticks, and grenades. Let's talk about each of them individually:


### Demo Header

The `header` attribute within a `Demo` object contains metadata pertaining to the parsed demofile. Metadata that can be found in the header includes map name, server information, etc. Below is a printed sample demo header for clarity:

In [16]:
for demo_header_key, demo_header_value in demo.header:
    print(demo_header_key + ":", demo_header_value)

demo_version_guid: 8e9d71ab-04a1-4c01-bb61-acfede27c046
network_protocol: 13985
fullpackets_version: 2
allow_clientside_particles: True
addons: 
client_name: SourceTV Demo
map_name: de_overpass
server_name: challengermode.com - Register to join
demo_version_name: valve_demo_2
allow_clientside_entities: True
demo_file_stamp: PBDEMS2 
game_directory: /home/dathost/cs2_linux/game/csgo


### Events

Events are emitted by the game server to keep all clients local states consistent. Events that are emitted by the server include kills, damages, item pickups and disconnects, to name a few. The events attribute of the demo object allows a user to access all occurrences of a particular event in a dataframe format. A round_id column will also be available in the DataFrames if `rounds` was set to `True` in `parse_demo`. Below is a breakdown of the different columns in a few different event types:

In [17]:
for event in demo.events:
    print(event)
    print("Columns:", ", ".join(demo.events[event]), "\n")

smokegrenade_expired
Columns: entityid, tick, user_name, user_steamid, x, y, z, event_type, round_id 

round_announce_match_start
Columns: tick, event_type, round_id 

smokegrenade_detonate
Columns: entityid, tick, user_name, user_steamid, x, y, z, event_type, round_id 

round_officially_ended
Columns: tick, event_type, round_id 

announce_phase_end
Columns: tick, event_type, round_id 

player_death
Columns: assistedflash, assister_name, assister_steamid, attacker_name, attacker_steamid, attackerblind, distance, dmg_armor, dmg_health, dominated, headshot, hitgroup, noreplay, noscope, penetrated, revenge, thrusmoke, tick, user_name, user_steamid, weapon, weapon_fauxitemid, weapon_itemid, weapon_originalowner_xuid, wipe, event_type, round_id 

begin_new_match
Columns: tick, event_type, round_id 

bomb_defused
Columns: site, tick, user_name, user_steamid, event_type, round_id 

round_end
Columns: legacy, message, nomusic, player_count, reason, tick, winner, event_type, round_end_reason, r

The above events are the default events parsed by `parse_demo`. If the optional argument `extended_events` is set as `True`, then additional events will be parsed. These extended events can be found within the method `build_event_list` in `awpy.parser.demoparser`.

#### Round-related events

There are a number of events outlined above but a certain subset we'd like to highlight are the round-related events. In particular, the events below:

* `round_start` - Signals the start of a new round as the gameclock starts ticking down while players stay frozen
* `round_freeze_end` - Signals the beginning of regulation time as the clock is set to 1:55 and players are unfrozen (the time before a round begins when player can access buy menu but their position is frozen)
* `round_end` - Winner for the current round is deterined, the current score for the match is updated, and alive players have the ability to move for a few seconds
* `round_officially_ended` - We move onto the next round and we should see a `round_start` event soon

### Ticks

A tick is when an update is sent to the server and propagated to the users. These "updates" (or ticks) happen many times a second and can be quantified by the tickrate. Like mentioned at the beginning of this notebook, the tickrate can differ based on gamemode and server; matchmaking uses 64-tick while professional games are played on 128-tick servers.

awpy provides parsed data organized by tick. If the `tick` argument is set to `True` in `parse_demo` (which it is by default) then the ticks attribute of the returned demo object will be populated with a DataFrame. This DataFrame will have a row for each player for each tick in the demo. This dataframe will also have information for **every tick** available through the demofile, so it is possible to have memory issues on certain machines/setups. The columns of the DataFrame are mostly player-related features (such as armor, health, active_weapon, etc.). A round_id column will also be available if `rounds` was set to `True` in `parse_demo`. The complete breakdown of the ticks DataFrame columns and a quick preview of the DataFrame are shown below:

In [19]:
demo.ticks.columns

Index(['game_phase', 'ping', 'armor', 'health', 'team_num', 'active_weapon',
       'has_defuser', 'has_helmet', 'current_equip_value',
       'round_start_equip_value', 'last_place_name', 'is_alive', 'pitch',
       'yaw', 'X', 'Y', 'Z', 'tick', 'steamid', 'name', 'round_id'],
      dtype='object')

In [20]:
demo.ticks.head(5)

,game_phase,ping,armor,health,team_num,active_weapon,has_defuser,has_helmet,current_equip_value,round_start_equip_value,...,is_alive,pitch,yaw,X,Y,Z,tick,steamid,name,round_id
469539,2,14,NaN,100.0,3.0,8192718.0,False,False,200.0,200.0,...,True,0.000000,160.000061,-2084.0,862.0,484.131256,79614,76561199063068840,w0nderful,1
469540,2,4,NaN,100.0,2.0,1704662.0,False,False,200.0,200.0,...,True,1.403503,51.606567,-1271.0,-3262.0,267.212341,79614,76561198080114546,fame,1
469541,2,13,NaN,100.0,3.0,3310271.0,False,False,200.0,200.0,...,True,0.000000,160.000061,-2359.0,875.0,476.131256,79614,76561198176878303,jL.,1
469542,2,11,NaN,100.0,3.0,11027164.0,False,False,200.0,200.0,...,True,0.000000,135.000000,-2190.0,817.0,476.665955,79614,76561198246607476,b1t,1
469543,2,13,NaN,100.0,3.0,13124314.0,False,False,200.0,200.0,...,True,0.000000,160.000061,-2164.0,900.0,484.131256,79614,76561198050250233,iM,1


If the argument `extended_ticks` for `parse_demo` is set as `True` (`False` by default), then additional information will be logged in the tick-level data. `extended_ticks` will create a larger set of columns in the ticks DataFrame. A few of the added columns are if the player is currently scoped, if the player is currently walking, etc. If you are interested in seeing the entire `extended_ticks` list, take a look at `build_tick_properties` in `demoparser.py`.

If the argument `keystrokes` for `parse_demo` is set as `True` (`False` by default), then additional information will be logged in the tick-level data. `keystrokes` allows one to see which buttons each player was clicking during each tick by adding certain buttons as columns to the ticks DataFrame. The tracked buttons are as follows: forward, backward, left, right, right click, reload, walk, zoom, scoreboard. Refer to `build_tick_properties` if you have any additional questions.

### Grenades

All instances of grenades (smoke, flashbang, molotov, and he_grenade) can also be organized in a DataFrame and stored in the `grenades` attribute of the demo object. There is a row for each tick a thrown grenade is active (smoke is bloomed, molotov is currently burning, etc.). So it is expected there are multiple rows representing the same grenade as grenades can be active over a certain timeframe, causing them to be active over multiple ticks. Below are the columns of the grenades DataFrame and a quick preview as well:

In [21]:
demo.grenades.columns

Index(['X', 'Y', 'Z', 'tick', 'thrower_steamid', 'name', 'grenade_type',
       'entity_id'],
      dtype='object')

In [22]:
demo.grenades.head(10)

,X,Y,Z,tick,thrower_steamid,name,grenade_type,entity_id
0,-2107.03125,982.09375,580.3125,81478,76561198013243326,AleksibOb,smoke,635
1,-2100.8125,974.8125,588.90625,81479,76561198013243326,AleksibOb,smoke,635
2,-2094.59375,967.5625,597.4375,81480,76561198013243326,AleksibOb,smoke,635
3,-2088.375,960.28125,605.90625,81481,76561198013243326,AleksibOb,smoke,635
4,-2082.15625,953.0,614.28125,81482,76561198013243326,AleksibOb,smoke,635
5,-2075.9375,945.75,622.5625,81483,76561198013243326,AleksibOb,smoke,635
6,-2069.71875,938.46875,630.78125,81484,76561198013243326,AleksibOb,smoke,635
7,-2063.5,931.1875,638.90625,81485,76561198013243326,AleksibOb,smoke,635
8,-2057.28125,923.9375,646.96875,81486,76561198013243326,AleksibOb,smoke,635
9,-2051.0625,916.65625,654.96875,81487,76561198013243326,AleksibOb,smoke,635
